# Hyundai Kite 

In [1]:
import pandas as pd
import numpy as np
# Import datetime from the datetime module
from datetime import datetime

## Data

### DW Hyundai

In [114]:
%%time
dw = pd.read_excel('data/DW HYUNDAI.xlsx')
dw.shape

Wall time: 30.2 s


(378324, 7)

In [115]:
dw = dw.astype(str)

In [122]:
dw.rename(columns = {'SIM_CARD': 'icc', 'LINHA': 'msisdn'}, inplace = True)
dw['msisdn'] = dw['msisdn'].str.replace("\.0", "")
dw['msisdn'] = "55" + dw['msisdn'].astype(str)

# > df['col'] = 'str' + df['col'].astype(str)

C:\Users\rodri\AppData\Local\Temp/ipykernel_8344/1361428635.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dw['msisdn'] = dw['msisdn'].str.replace("\.0", "")


In [162]:
# dw.head()

### Removing duplicated iccid

In [124]:
dw.drop_duplicates(subset=['icc'], keep='first', inplace=True)
dw.shape

(68707, 7)

### Kite data

In [125]:
def getDataFromKite(csvFile):
    df = pd.read_csv(csvFile, sep=';', low_memory=False)
    df = df[['icc', 'imsi', 'msisdn']]
    df = df.applymap(str) # change all columns to string type
    df = df.applymap(lambda x: x.lstrip('=""').rstrip('"')) # remove unnecessary characteres
    
    return df

In [126]:
%%time
kite_df = getDataFromKite('data/SimInventoryExport_2022-06-24T15.30.21Z_pzkhe9-2kbeqrc2saf-62b5d88c98f29.0.csv')
kite_df = kite_df.astype(str)
kite_df.shape

Wall time: 4.56 s


(132112, 3)

In [161]:
# kite_df.head()

In [128]:
# kite_df.info()

## Comparing data using ICC

In [129]:
dw.shape, kite_df.shape

((68707, 7), (132112, 3))

### Data in both Kite and DW

In [130]:
# df_inner = dw.merge(kite_df, on='icc', how='inner')
# df_inner.shape

kite_and_dw = kite_df[ (kite_df['icc'].isin(dw['icc'])) ]
kite_and_dw.shape

(68290, 3)

### Data only in Kite and not in DW

In [131]:
only_kite = kite_df[ ~(kite_df['icc'].isin(dw['icc'])) ]
only_kite.shape

(63822, 3)

In [132]:
kite_df.shape[0] == kite_and_dw.shape[0] + only_kite.shape[0]

True

### Data only in DW

In [144]:
only_dw = dw[ ~(dw['icc'].isin(kite_df['icc'])) ]
only_dw.shape

(417, 7)

In [146]:
dw.shape[0] == kite_and_dw.shape[0] + only_dw.shape[0]

True

### Export

In [157]:
kite_and_dw.to_excel("data/kite_and_dw_24.06.2022.xlsx")

In [158]:
only_dw.to_excel("data/only_dw_24.06.2022.xlsx")

In [159]:
only_kite.to_excel("data/only_kite_24.06.2022.xlsx")

## Comparing using msisdn

### Both kite and dw

In [139]:
kite_and_dw_msisdn = kite_df[ (kite_df['msisdn'].isin(dw['msisdn'])) ]
kite_and_dw_msisdn.shape

(68283, 3)

### Data only in Kite and not in DW

In [140]:
only_kite_msisdn = kite_df[ ~(kite_df['msisdn'].isin(dw['msisdn'])) ]
only_kite_msisdn.shape

(63829, 3)

In [142]:
kite_df.shape[0] == kite_and_dw_msisdn.shape[0] + only_kite_msisdn.shape[0]

True

### Data only in DW

In [148]:
only_dw_msisdn = dw[ ~(dw['msisdn'].isin(kite_df['msisdn'])) ]
only_dw_msisdn.shape

(424, 7)

In [149]:
dw.shape[0] == kite_and_dw_msisdn.shape[0] + only_dw_msisdn.shape[0]

True

## ICC vs MSISDN

In [156]:
kite_and_dw.shape[0] - kite_and_dw_msisdn.shape[0]

7

In [160]:
# kite_and_dw[ ~(kite_and_dw['msisdn'].isin(kite_and_dw_msisdn['msisdn'])) ]